# Лабораторна робота 3 з Симетричної Криптографії

**Варіант:** 2

**Команда:** Бондар, Кістаєв

**Група:** ФІ-03

## Підготовча частина: оголошення констант, функція для зчитування і передобробки тексту

In [ ]:
# Constants
A1 = "абвгдежзийклмнопрстуфхцчшщыьэюя"
A2 = "абвгдежзийклмнопрстуфхцчшщьыэюя"
ALPH = A1
m = len(ALPH)
M = m*m
FIVE_MOST_FREQUENT = ['ст', 'но', 'то', 'на', 'ен']

RING = {}
BIGRAM_RING = {}
BIGRAMS = ['aa'] * (M)

# Precompute transformations
for char in ALPH:
    pos = ALPH.index(char)
    RING[char] = pos

for i in ALPH:
    for j in ALPH:
        pos_i = ALPH.index(i)
        pos_j = ALPH.index(j)
        idx = pos_i * m + pos_j
        BIGRAM_RING[i+j] = idx
        BIGRAMS[idx] = i+j

TEST_TEXT = "test_text.txt"
FILE_TO_ENCRYPT = "text_to_encrypt.txt"

# print(BIGRAMS)
# print(BIGRAM_RING)

In [ ]:
# Text preprocessing
def transform_symbol(_c):
    if 'а' <= _c and _c <= 'я':
        return _c
    elif _c <= 'Я' and _c >= 'А':
        return _c.lower()
    elif _c == 'Ё' or _c == 'ё':
        return 'е'
    else:
        return ""
    
def preprocess_text(_text):
    text_formatted = ""
    # Change symbols according to requirements
    for c in _text:
        text_formatted += transform_symbol(c)
    # Remove consequtive spaces
    text_formatted = ' '.join(text_formatted.split())
    return text_formatted

def read_text(filename):
    f = open(filename, "r", encoding='utf-8')
    text = f.read()
    f.close()
    return preprocess_text(text)

## Додатковий функціонал для атаки: РАЕ, розв'язування лінійних порівнянь

In [ ]:
def extended_euclidian(a, b):
    r_0, r_1 = a, b
    u_0, u_1 = 1, 0 # coef at a
    v_0, v_1 = 0, 1 # coef at b

    # Parallel computation of remainders and coefs
    while r_1 != 0:
        q = r_0 // r_1
        r_0, r_1 = r_1, r_0 - q * r_1   # same as r_1, r_0 % r_1
        u_0, u_1 = u_1, u_0 - q * u_1
        v_0, v_1 = v_1, v_0 - q * v_1

    # print(f"gcd({a}, {b}) = {r_0} = {u_0}*{a} + {v_0}*{b}")
    return (r_0, u_0, v_0)

In [230]:
# ax = b (mod n) -> list of possible solutions
def solve_linear_congruence(a, b, n):
    a %= n
    b %= n
    # print(f"{a}x = {b} (mod {n})")
    (d, a_coef, _) = extended_euclidian(a, n) # Check gcd(a, n)
    if d == 1:
        # then a^-1 = a_coef
        return [(a_coef * b) % n]
    elif b % d == 0:
        a_1, b_1, n_1 = a // d, b // d, n // d
        (_, a_1_inv, _) = extended_euclidian(a_1, n_1)
        x_0 = (a_1_inv * b_1) % n_1
        return [x_0 + k * n_1 for k in range(d)]
    else:
        return []

SyntaxError: 'tuple' is an illegal expression for augmented assignment (3855590675.py, line 11)

## Аналіз шифротексту: підрахунок біграм, знаходження n найчастіших біграм у тексті.
При застосуванні алгоритму, використовуватимемо n = 5.

In [ ]:
# Bigrams without intersection (ex: [1, 2], [3, 4])
def count_bigrams_wo_i(_text):
    b_count = {}
    i = 1
    while i < len(_text):
        bg = _text[i - 1] + _text[i]
        if bg not in b_count:
            b_count[bg] = 1
        else:
            b_count[bg] = b_count[bg] + 1 
        i = i + 2
    # Sort by amount reversed
    return dict(sorted(b_count.items(), key=lambda pair: -pair[1]))

def get_best_n_bigrams(_text, n):
    bigrams_count = count_bigrams_wo_i(_text)
    return list(bigrams_count)[:n:]

## Отримання ключа, дешифрування тексту за допомогою цього ключа

In [ ]:
def get_keys(best_5):
    keys = []
    for i_1 in range(5):
        for j_1 in range(5):
            for i_2 in range(5):
                for j_2 in range(5):
                    if i_1 == i_2 or j_1 == j_2:
                        continue
                    
                    X_1 = BIGRAM_RING[FIVE_MOST_FREQUENT[i_1]]
                    Y_1 = BIGRAM_RING[best_5[j_1]]
                    X_2 = BIGRAM_RING[FIVE_MOST_FREQUENT[i_2]]
                    Y_2 = BIGRAM_RING[best_5[j_2]]

                    A = solve_linear_congruence(X_1 - X_2, Y_1 - Y_2, M)
                    keys += [(a, (Y_1 - a * X_1) % M) for a in A]
    return keys

In [ ]:
def decrypt_bigram(encr, a_inv, b):
    Y = BIGRAM_RING[encr]
    return BIGRAMS[a_inv * (Y - b) % M]

def try_decrypt_text(_text, key):
    (a, b) = key
    (d, a_inv, _) = extended_euclidian(a, M)
    if d != 1:
        return "invalid key!"
    
    res = ""
    for i in range(1, len(_text), 2):
        encr = _text[i - 1] + _text[i]
        res += decrypt_bigram(encr, a_inv, b)
    
    return res

## Перевірка тексту на змістовність

## Побудова атаки

In [ ]:
text = read_text(TEST_TEXT)
best_5 = get_best_n_bigrams(text, 5)

keys = get_keys(best_5)
for key in keys:
    open_text = try_decrypt_text(text, key)
    if open_text == "invalid key!":
        continue
    print(f"-------------------{key}-------------------")
    print(open_text)